In [34]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import logging as log
import sys
%matplotlib inline

In [35]:
def logging_setup(logging):
    # Color for warning and error mesages
    logging.addLevelName(
        logging.WARNING,
        "\033[1;33m%s\033[1;0m" % logging.getLevelName(logging.WARNING))
    logging.addLevelName(
        logging.ERROR,
        "\033[1;31m%s\033[1;0m" % logging.getLevelName(logging.ERROR))

    logging.basicConfig(
        format='%(levelname)s: %(message)s',
        stream=sys.stderr,
        level=logging.INFO)
logging_setup(log)

In [36]:
import json
import os

print(os.getcwd())
DIR = "../aetherling/resource-cmp"
aetherling = json.load(open(DIR + "/chisel-conv-16.json"))
aetherling['name'] = 'aetherling'
filament = json.load(open(DIR + "/fil-con.json"))
filament['name'] = 'filament'
fil_reticle = json.load(open(DIR + "/fil-reticle-conv.json"))
fil_reticle['name'] = 'filament-reticle'


/Users/rachitnigam/git/filament/scripts


In [70]:
def merge_jsons(*jsons):
    dfs = [pd.json_normalize(j) for j in jsons]
    return pd.concat(dfs)

all_data = merge_jsons(aetherling, filament, fil_reticle).set_index('name')
all_data

,lut,dsp,registers,muxes,clb_registers,carry8,f7_muxes,f8_muxes,f9_muxes,clb,...,worst_slack,period,frequency,cell_lut1,cell_lut2,cell_lut3,cell_lut4,cell_lut5,cell_lut6,cell_fdre
name,,,,,,,,,,,,,,,,,,,,,
aetherling,106,10,78,2,236,8,0,0,0,350,...,0.160,1.5,666.667,5,72,2,1,1,-1,236
filament,128,9,11,0,88,16,0,0,0,232,...,0.045,1.2,833.333,-1,128,-1,-1,-1,-1,88
filament-reticle,14,9,20,0,160,0,0,0,0,174,...,0.076,1.6,625.000,-1,3,3,-1,6,6,160


In [79]:
FIELDS = ['lut', 'dsp', 'registers', 'muxes', 'clb_registers', 'frequency', 'worst_slack']
MIN_SLACK = 0.1
# Remove all the rows that only contain zero values
def cleanup(df):
    # Error if any design didn't meet timing
    for (name, meets) in df['meet_timing'].items():
        if meets == 0:
            log.error(f"{name} did not meet timing")

    # If the slack is less than the minimum
    for (name, slack) in df['worst_slack'].items():
        if slack > MIN_SLACK:
            log.warning(f"{name} has slack {slack} which is more than the minimum {MIN_SLACK}")

    # Make all columns print 2 decimal places
    pd.options.display.float_format = '{:,.2f}'.format

    df = df[FIELDS]

    # Remap names of columns
    df = df.rename(columns={
        'lut': 'LUTs',
        'dsp': 'DSPs',
        'registers': 'Registers',
        'muxes': 'Muxes',
        'clb_registers': 'CLB Registers',
        'frequency': 'Frequency (MHz)',
        'worst_slack': 'Worst Slack (ns)',
    })
    
    return df


cleanup(all_data)

,LUTs,DSPs,Registers,Muxes,CLB Registers,Frequency (MHz),Worst Slack (ns)
name,,,,,,,
aetherling,106,10,78,2,236,666.67,0.16
filament,128,9,11,0,88,833.33,0.04
filament-reticle,14,9,20,0,160,625.00,0.08
